In [1]:
import os
import re
import sqlite3
import pandas as pd
from pathlib import Path
import glob



In [2]:
def normalize_name(s: str) -> str:
    s = s.strip()
    s = re.sub(r"\s+", "_", s)
    s = re.sub(r"[^\w]", "_", s)
    s = re.sub(r"_+", "_", s)
    s = s.strip("_")
    if not s:
        s = "col"
    # SQLite 虽然不强制，但建议不要以数字开头
    if s[0].isdigit():
        s = f"t_{s}"
    return s

def table_name_from_file(fp: str) -> str:
    base = os.path.splitext(os.path.basename(fp))[0]
    return normalize_name(base)


In [3]:
csv_dir = Path("Data/CISS/CSV_files")          # 放csv的文件夹
db_path = Path("Data/CISS/SqlDatabase/CISS.sqlite")    # sqlite文件路径



csv_files = sorted(glob.glob(os.path.join(csv_dir, "*.csv")))

conn = sqlite3.connect(db_path)

for fp in csv_files:
    table = table_name_from_file(fp)

    # 读取（常见：UTF-8 / UTF-8-SIG；如果你是中文 Windows CSV，有时是 gbk）
    try:
        df = pd.read_csv(fp,encoding="gbk")
    except UnicodeDecodeError:
        df = pd.read_csv(fp, encoding="utf-8-sig")

    # 清洗列名（避免空格、特殊符号导致后续 SQL 麻烦）
    df.columns = [normalize_name(c) for c in df.columns]

    # 可选：把全空列丢掉
    # df = df.dropna(axis=1, how="all")

    # 写入 SQLite（存在就替换；想追加用 if_exists="append"）
    df.to_sql(table, conn, if_exists="replace", index=False)

    print(f"✅ Imported {os.path.basename(fp)} -> table: {table} (rows={len(df)})")

conn.close()
print(f"🎉 Done. SQLite saved at: {db_path}")


✅ Imported AVOID.csv -> table: AVOID (rows=217902)
✅ Imported CRASH.csv -> table: CRASH (rows=21456)
✅ Imported EVENT.csv -> table: EVENT (rows=40675)
✅ Imported GV.csv -> table: GV (rows=38121)
✅ Imported OCC.csv -> table: OCC (rows=42706)
✅ Imported OCCONTACT.csv -> table: OCCONTACT (rows=48581)
✅ Imported VPICDECODE.csv -> table: VPICDECODE (rows=24113)
🎉 Done. SQLite saved at: Data/CISS/SqlDatabase/CISS.sqlite


In [4]:
conn = sqlite3.connect(db_path)
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;",
    conn
)
print(tables)
conn.close()

         name
0       AVOID
1       CRASH
2       EVENT
3          GV
4         OCC
5   OCCONTACT
6  VPICDECODE
